In [1]:
!pip install --no-index --find-links /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl pytorch-tabnet

Looking in links: /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl
Processing /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import tensorflow as tf
import tensorflow_addons as tfa
import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from pytorch_tabnet.tab_model import TabNetRegressor
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn import functional as F

/opt/conda/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:68: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.3.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [3]:
x_train = pd.read_csv('../input/lish-moa/train_features.csv')
x_test = pd.read_csv('../input/lish-moa/test_features.csv')
y_train = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
y_train_non = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [4]:
y_train_non.shape

(23814, 403)

In [5]:
trt_cp_index=x_train[x_train['cp_type']=='trt_cp'].index

In [6]:
x_test_checkpoint=x_test.copy()

In [7]:
x_test_checkpoint.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_0004d9e33,trt_cp,24,D1,-0.5458,0.1306,-0.5135,0.4408,1.5500,-0.1644,...,0.0981,0.7978,-0.1430,-0.2067,-0.2303,-0.1193,0.0210,-0.0502,0.1510,-0.7750
1,id_001897cda,trt_cp,72,D1,-0.1829,0.2320,1.2080,-0.4522,-0.3652,-0.3319,...,-0.1190,-0.1852,-1.0310,-1.3670,-0.3690,-0.5382,0.0359,-0.4764,-1.3810,-0.7300
2,id_002429b5b,ctl_vehicle,24,D1,0.1852,-0.1404,-0.3911,0.1310,-1.4380,0.2455,...,-0.2261,0.3370,-1.3840,0.8604,-1.9530,-1.0140,0.8662,1.0160,0.4924,-0.1942
3,id_00276f245,trt_cp,24,D2,0.4828,0.1955,0.3825,0.4244,-0.5855,-1.2020,...,0.1260,0.1570,-0.1784,-1.1200,-0.4325,-0.9005,0.8131,-0.1305,0.5645,-0.5809
4,id_0027f1083,trt_cp,48,D1,-0.3979,-1.2680,1.9130,0.2057,-0.5864,-0.0166,...,0.4965,0.7578,-0.1580,1.0510,0.5742,1.0900,-0.2962,-0.5313,0.9931,1.8380


In [8]:
x_train.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [9]:
x_train=x_train.drop(['sig_id'],1)
x_test=x_test.drop(['sig_id'],1)
y_train=y_train.drop(['sig_id'],1)
x_train.head()

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [10]:
x_train=x_train.drop(['cp_type'],1)
x_test=x_test.drop(['cp_type'],1)

In [11]:
x_train['cp_time'] = x_train['cp_time'].map({24:0,48:1,72:2})
x_train.cp_time.unique()
x_test['cp_time'] = x_test['cp_time'].map({24:0,48:1,72:2})
x_test.cp_time.unique()

array([0, 2, 1])

In [12]:
x_train['cp_dose'] = x_train['cp_dose'].map({"D1":0,"D2":1})
x_train.cp_dose.unique()
x_test['cp_dose'] = x_test['cp_dose'].map({"D1":0,"D2":1})
x_test.cp_dose.unique()

array([0, 1])

In [13]:
x_train

,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,0,0,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,-0.0326,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,2,0,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,0.3372,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,1,0,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,0.2155,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,1,0,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,0.1792,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,2,1,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,-0.1498,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,0,1,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,0.5773,0.3055,...,0.1969,0.0262,-0.8121,0.3434,0.5372,-0.3246,0.0631,0.9171,0.5258,0.4680
23810,0,1,-1.3260,0.3478,-0.3743,0.9905,-0.7178,0.6621,-0.2252,-0.5565,...,0.4286,0.4426,0.0423,-0.3195,-0.8086,-0.9798,-0.2084,-0.1224,-0.2715,0.3689
23811,1,1,0.3942,0.3756,0.3109,-0.7389,0.5505,-0.0159,-0.2541,0.1745,...,0.5409,0.3755,0.7343,0.2807,0.4116,0.6422,0.2256,0.7592,0.6656,0.3808
23812,0,0,0.6660,0.2324,0.4392,0.2044,0.8531,-0.0343,0.0323,0.0463,...,-0.1105,0.4258,-0.2012,0.1506,1.5230,0.7101,0.1732,0.7015,-0.6290,0.0740


In [14]:
genes=[col for col in x_train.columns if col.startswith("g-")]
cells=[col for col in x_train.columns if col.startswith("c-")]

In [15]:
a_train=x_train.copy()
a_test=x_test.copy()

In [16]:
x_train=a_train.copy()
x_test=a_test.copy()

In [17]:
scaler = QuantileTransformer(n_quantiles=100,random_state=42, output_distribution="normal")
scaled_data=scaler.fit_transform(x_train[genes+cells])
x_train=pd.concat([x_train.iloc[:,:2],pd.DataFrame(scaled_data,columns=genes+cells)],1)
# scaled_data=scaler.transform(x_test[genes+cells])
# x_test=pd.concat([x_test.iloc[:,:2],pd.DataFrame(scaled_data,columns=genes+cells)],1)


transformer = QuantileTransformer(n_quantiles=100,random_state=42, output_distribution="normal")
scaled_data=scaler.fit_transform(x_test[genes+cells])
x_test=pd.concat([x_test.iloc[:,:2],pd.DataFrame(scaled_data,columns=genes+cells)],1)

In [18]:
x_test.iloc[:,2:].sum().sum()

-8977.505198164718

In [19]:
# genes = [cols for cols in x_train.columns if cols.startswith('c-')]
# data=pd.concat([pd.DataFrame(x_train[genes]),pd.DataFrame(x_test[genes])])
# g_data=PCA(0.95,random_state=42).fit_transform(data)

In [20]:
# g_data.sum()

In [21]:
data=pd.concat((x_train[genes],x_test[genes]))
g_data=PCA(n_components=0.95,random_state=44).fit_transform(data)

g_train=g_data[:x_train.shape[0]]
g_test=g_data[x_train.shape[0]:]

g_train_df=pd.DataFrame(g_train,columns=[f'pca_g-{i}' for i in range(g_data.shape[1])])
g_test_df=pd.DataFrame(g_test,columns=[f'pca_g-{i}' for i in range(g_data.shape[1])])


data=pd.concat((x_train[cells],x_test[cells]))
c_data=PCA(n_components=0.95,random_state=44).fit_transform(data)

c_train=c_data[:x_train.shape[0]]
c_test=c_data[x_train.shape[0]:]

c_train_df=pd.DataFrame(c_train,columns=[f'pca_c-{i}' for i in range(c_data.shape[1])])
c_test_df=pd.DataFrame(c_test,columns=[f'pca_c-{i}' for i in range(c_data.shape[1])])

In [22]:
X_train=pd.concat((x_train,g_train_df,c_train_df),axis=1)
X_test=pd.concat((x_test,g_test_df,c_test_df),axis=1)

In [23]:
from sklearn.feature_selection import VarianceThreshold
varthre = VarianceThreshold(0.8)
train_varthre=varthre.fit_transform(X_train.iloc[:,2:])
test_varthre=varthre.transform(X_test.iloc[:,2:])

In [24]:
X_train=pd.concat([X_train.iloc[:,:2],pd.DataFrame(train_varthre,columns=X_train.columns[2:][varthre.get_support()])],1)
X_test=pd.concat([X_test.iloc[:,:2],pd.DataFrame(test_varthre,columns=X_test.columns[2:][varthre.get_support()])],1)

X_train.shape,X_test.shape,y_train.shape,submission.shape

((23814, 1038), (3982, 1038), (23814, 206), (3982, 207))

In [25]:
display(X_train.head(3))
print(X_train.shape)
display(X_test.head(3))
print(X_test.shape)

,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,...,pca_g-154,pca_g-155,pca_c-0,pca_c-1,pca_c-2,pca_c-3,pca_c-4,pca_c-5,pca_c-6,pca_c-7
0,0,0,1.134849,0.907687,-0.416385,-0.966814,-0.254723,-1.017473,-1.364787,-0.024007,...,0.003567,1.595561,4.917651,1.538645,-1.560099,1.127068,0.908941,-1.080358,-0.253258,0.115213
1,2,0,0.119282,0.681738,0.272399,0.080113,1.205169,0.686517,0.313396,0.555877,...,-1.268543,-0.743488,5.090695,-0.368828,0.013911,0.994664,-0.502353,0.673057,-0.272862,0.455034
2,1,0,0.779973,0.946463,1.425350,-0.132928,-0.006122,1.492493,0.235577,0.365511,...,-0.876965,-0.960548,-1.352416,0.294769,-0.291118,-0.218369,-0.098231,-0.711736,0.893185,0.039182


(23814, 1038)


,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,...,pca_g-154,pca_g-155,pca_c-0,pca_c-1,pca_c-2,pca_c-3,pca_c-4,pca_c-5,pca_c-6,pca_c-7
0,0,0,-0.755294,0.214796,-0.774511,0.705349,1.564580,-0.194968,-0.276944,0.363393,...,0.494774,0.372851,-0.804259,-1.899650,0.657893,-0.338125,0.099395,-0.800232,0.523428,-0.393640
1,2,0,-0.186773,0.374643,1.176402,-0.652299,-0.546638,-0.403447,-2.325903,0.645634,...,-0.610010,0.790070,-5.397308,-0.456508,0.290936,-1.095456,-0.612847,1.042001,0.832567,1.149697
2,0,0,0.351229,-0.155308,-0.613250,0.255053,-1.762152,0.333321,-0.434423,-0.484560,...,-0.026030,-0.539568,3.589625,1.289070,0.156473,-0.164602,-1.030081,2.044905,-0.786719,-0.974288


(3982, 1038)


In [26]:
genes=[col for col in X_train.columns if col.startswith("g-")]
cells=[col for col in X_train.columns if col.startswith("c-")]

In [27]:
from sklearn.cluster import KMeans
# kmeans=KMeans(n_clusters=40,random_state=982)
kmeans=KMeans(n_clusters=35,random_state=982)
kmeans.fit(pd.concat([X_train[genes],X_test[genes]]))
X_train["cluster_g"]=kmeans.labels_[:len(X_train)]
X_test["cluster_g"]=kmeans.labels_[len(X_train):]

from sklearn.cluster import KMeans
# kmeans=KMeans(n_clusters=8,random_state=982)
kmeans=KMeans(n_clusters=5,random_state=982)
kmeans.fit(pd.concat([X_train[cells],X_test[cells]]))
X_train["cluster_c"]=kmeans.labels_[:len(X_train)]
X_test["cluster_c"]=kmeans.labels_[len(X_train):]

X_train=pd.get_dummies(X_train,columns=['cluster_g','cluster_c'])
X_test=pd.get_dummies(X_test,columns=['cluster_g','cluster_c'])

In [28]:
gsquarecols=['g-574','g-211','g-216','g-0','g-255','g-577','g-153','g-389','g-60','g-370','g-248','g-167','g-203','g-177','g-301','g-332','g-517','g-6','g-744','g-224','g-162','g-3','g-736','g-486','g-283','g-22','g-359','g-361','g-440','g-335','g-106','g-307','g-745','g-146','g-416','g-298','g-666','g-91','g-17','g-549','g-145','g-157','g-768','g-568','g-396']

In [29]:
for df in X_train,X_test:
    df['time_dose']=df['cp_time']+df['cp_dose']
    df['g_median']=df[genes].median(axis=1)
    df['c_median']=df[cells].median(axis=1)
    df['gc_median']=df[genes+cells].median(axis=1)
    df['g_max']=df[genes].max(axis=1)
    df['c_max']=df[cells].max(axis=1)
    df['gc_max']=df[genes+cells].max(axis=1)
    df['g_min']=df[genes].min(axis=1)
    df['c_min']=df[cells].min(axis=1)
    df['gc_min']=df[genes+cells].min(axis=1)
    df['g_sum']=df[genes].sum(axis=1)
    df['c_sum']=df[cells].sum(axis=1)
    df['gc_sum']=df[genes+cells].sum(axis=1)
    df['g_mean']=df[genes].mean(axis=1)
    df['c_mean']=df[cells].mean(axis=1)
    df['gc_mean']=df[genes+cells].mean(axis=1)
    df['g_std']=df[genes].std(axis=1)
    df['c_std']=df[cells].std(axis=1)
    df['gc_std']=df[genes+cells].std(axis=1)
    df['g_skew']=df[genes].skew(axis=1)
    df['c_skew']=df[cells].skew(axis=1)
    df['gc_skew']=df[genes+cells].skew(axis=1)
    df['g_kurtosis']=df[genes].kurtosis(axis=1)
    df['c_kurtosis']=df[cells].kurtosis(axis=1)
    df['gc_kurtosis']=df[genes+cells].kurtosis(axis=1)
    df['g_q25']=df[genes].quantile(0.25,axis=1)
    df['c_q25']=df[cells].quantile(0.25,axis=1)
    df['gc_q25']=df[genes+cells].quantile(0.25,axis=1)
    df['g_q75']=df[genes].quantile(0.75,axis=1)
    df['c_q75']=df[cells].quantile(0.75,axis=1)
    df['gc_q75']=df[genes+cells].quantile(0.75,axis=1)
    
    
    df['c52_c42'] = df['c-52'] * df['c-42']
    df['c13_c73'] = df['c-13'] * df['c-73']
    df['c26_c13'] = df['c-23'] * df['c-13']
    df['c33_c6'] = df['c-33'] * df['c-6']
    df['c11_c55'] = df['c-11'] * df['c-55']
    df['c38_c63'] = df['c-38'] * df['c-63']
    df['c38_c94'] = df['c-38'] * df['c-94']
    df['c13_c94'] = df['c-13'] * df['c-94']
    df['c4_c52'] = df['c-4'] * df['c-52']
    df['c4_c42'] = df['c-4'] * df['c-42']
    df['c13_c38'] = df['c-13'] * df['c-38']
    df['c55_c2'] = df['c-55'] * df['c-2']
    df['c55_c4'] = df['c-55'] * df['c-4']
    df['c4_c13'] = df['c-4'] * df['c-13']
    df['c82_c42'] = df['c-82'] * df['c-42']
    df['c66_c42'] = df['c-66'] * df['c-42']
    df['c6_c38'] = df['c-6'] * df['c-38']
    df['c2_c13'] = df['c-2'] * df['c-13']
    df['c62_c42'] = df['c-62'] * df['c-42']
    df['c90_c55'] = df['c-90'] * df['c-55']
    
    for feature in cells:
             df[f'{feature}_squared'] = df[feature] ** 2     
                
    for feature in gsquarecols:
        df[f'{feature}_squared'] = df[feature] ** 2

In [30]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

(23814, 1274)
(3982, 1274)
(23814, 206)


In [31]:
X_train=X_train.loc[trt_cp_index,:]
y_train=y_train.loc[trt_cp_index,:]
y_train_non=y_train_non.loc[trt_cp_index,:]

In [32]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_train_non.shape)

(21948, 1274)
(3982, 1274)
(21948, 206)
(21948, 403)


In [33]:
# def getmodel(layers,dropouts,activations1,activations2,layer_sizes1,layer_sizes2,lr,wd):
#     model=tf.keras.Sequential()
#     model.add(tf.keras.Input(shape=(689,)))
    
    
#     for i in range(layers):
#         model.add(
#             tf.keras.layers.BatchNormalization()
#         )
#         model.add(
#             tf.keras.layers.Dropout(dropouts[i])
#         )
#         model.add(
#             tfa.layers.WeightNormalization(tf.keras.layers.Dense(layer_sizes1[i],activation=activations1[i]))
#         )
#         model.add(
#             tf.keras.layers.Dense(layer_sizes2[i],activation=activations2[i])
#         )
        
        
#     model.add(tf.keras.layers.Dense(206,activation='softmax'))
    
# #     model.compile(optimizer=tfa.optimizers.AdamW(lr = 1e-3, weight_decay = 1e-5, clipvalue = 756),loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])
#     model.compile(optimizer=tfa.optimizers.AdamW(lr = lr, weight_decay = wd, clipvalue = 756),loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])
    
#     return model


# def get_model(layers,units,dropouts,activations):
#     model=tf.keras.Sequential()
#     model.add(tf.keras.Input(shape=(874,)))
#     model.add(tf.keras.layers.BatchNormalization())
        
#     for i in range(layers):
#         model.add(tf.keras.layers.Dense(units[i], activation=activations[i]))
#         model.add(tf.keras.layers.Dropout(dropouts[i]))
#         model.add(tf.keras.layers.BatchNormalization())
            
#     model.add(tf.keras.layers.Dense(206,activation='softmax'))
    
#     model.compile(optimizer = tf.keras.optimizers.Adam(1e-3),loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
#     return model

def create_model(num_cols, hid_layers, activations, dropout_rate, lr, num_cols_y):
    
    inp1 = tf.keras.layers.Input(shape = (num_cols, ))
    x1 = tf.keras.layers.BatchNormalization()(inp1)

    for i, units in enumerate(hid_layers):
#         x1 = tfa.layers.WeightNormalization(tf.keras.layers.Dense(units, activation=activations[i]))(x1)
        x1 = tf.keras.layers.Dense(units, activation=activations[i])(x1)
        x1 = tf.keras.layers.Dropout(dropout_rate[i])(x1)
        x1 = tf.keras.layers.BatchNormalization()(x1)
    
    x1 = tf.keras.layers.Dense(num_cols_y,activation='sigmoid')(x1)
    model = tf.keras.models.Model(inputs= inp1, outputs= x1)
    
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=lr),loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.001), metrics=['accuracy'])
#     model.compile(optimizer=tfa.optimizers.AdamW(lr = 1e-3, weight_decay = 1e-5, clipvalue = 756),loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])
    
    return model

In [34]:
hps={
    30:[
    {
        "units":[1410,1246,512,512,512],
        "activations":['selu', 'swish', 'elu','elu','elu'],
        "dropouts":[0.49,0.51,0.0,0.0,0.0]
    },
    {
        "units":[1127,1133, 1209, 512],
        "activations":[ 'selu','swish','swish','elu'],
        "dropouts":[0.32,0.02,0.26,0.0]
    },
    {
        "units":[796,1995,573],
        "activations":['selu','selu','swish'],
        "dropouts":[0.22,0.21,0.09]
    },
    {
        "units":[959,1557,1826],
        "activations":['selu','selu','elu'],
        "dropouts":[0.42,0.61,0.3]
    },
    {
        "units":[1081,2033],
        "activations":['elu','elu'],
        "dropouts":[0.44,0.12]
    }    
    ],
    58:[
        {
            "units":[561,882,2023],
            "activations":['elu','elu','elu'],
            "dropouts":[0.43,0.17,0.29]
        },
        {
            "units":[687,1085,1365,1427,1325],
            "activations":['selu','swish','swish','elu','selu'],
            "dropouts":[0.59,0.15,0.31,0.08,0.41]
        },
        {
            "units":[1324,1475,976,512],
            "activations":['selu','selu','selu', 'elu'],
            "dropouts":[0.39,0.38,0.19,0.0]
        },
        {
            "units":[1410,1246,512,512,512],
            "activations":['selu', 'swish', 'elu', 'elu', 'elu'],
            "dropouts":[0.49,0.51,0.0,0.0,0.0]
        },
        {
            "units":[750,1734,638,512],
            "activations":['selu','swish','selu','elu'],
            "dropouts":[0.42,0.5,0.42,0.0]
        }
    ],
    3255:[
        {
            "units":[1444,629,512,512,512],
            "activations":['selu', 'swish', 'elu', 'elu', 'elu'],
            "dropouts":[0.62,0.4,0.0,0.0,0.0]
        },
        {
            "units":[899,940,893,718],
            "activations":['elu', 'elu', 'selu', 'selu'],
            "dropouts":[0.33,0.28,0.29,0.17]
        },
        {
            "units":[2030,1422],
            "activations":['elu', 'selu'],
            "dropouts":[0.53,0.37]
        },
        {
            "units":[1194,1562],
            "activations":[ 'swish','swish'],
            "dropouts":[0.59,0.0]
        },
        {
            "units":[1130,766,520,512],
            "activations":[  'selu','elu','elu', 'elu'],
            "dropouts":[0.29,0.14,0.27,0.0]
        }
    ]
}


In [35]:
from pickle import load,dump

In [36]:
y_train_non=y_train_non.iloc[:,y_train_non.columns!='sig_id']
final_y=pd.concat([y_train,y_train_non],1)
final_pred=pd.DataFrame(np.zeros((submission.shape[0], final_y.shape[1])),columns=final_y.columns)

for seed in [30,58,3255]:
    for n,(tr,te) in enumerate(MultilabelStratifiedKFold(n_splits=5, random_state=seed, shuffle=True).split(X_train,y_train)):
        print("SEED:"+str(seed)+" Fold:"+str(n))
        

        units=hps[seed][n]['units']
        activations=hps[seed][n]['activations']
        dropouts=hps[seed][n]['dropouts']
        
        model=create_model(X_train.shape[1], units, activations, dropouts, 1e-3, final_y.shape[1])
        

        model.fit(X_train.values[tr],
                 final_y.values[tr],
                 batch_size=128,
                 epochs=150,
                 validation_data=(X_train.values[te],final_y.values[te]),
                 verbose=1,
                  callbacks=[
                      tf.keras.callbacks.ReduceLROnPlateau(
                          monitor='val_loss', 
                          factor=0.1, 
                          patience=3,
                          epsilon = 1e-4, 
                          mode = 'min',
                          verbose=1
                      )
                      ,

                      tf.keras.callbacks.EarlyStopping(
                          monitor='val_loss',
                          min_delta=0,
                          patience=10,
                          mode='auto',
                          verbose=1,
                          baseline=None,
                          restore_best_weights=True
                      )
                  ]
                 )
        model.save(f'pSEED{seed}FOLD{n}.h5')
        model=create_model(X_train.shape[1], units, activations, dropouts, 1e-3, final_y.shape[1])
        model.load_weights(f'pSEED{seed}FOLD{n}.h5')
        final_pred+=model.predict(X_test)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=30 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


SEED:30 Fold:0
Epoch 1/150
138/138 [==============================] - 2s 12ms/step - loss: 0.2900 - accuracy: 0.0302 - val_loss: 0.0233 - val_accuracy: 0.0483
Epoch 2/150
138/138 [==============================] - 1s 10ms/step - loss: 0.0168 - accuracy: 0.0415 - val_loss: 0.0145 - val_accuracy: 0.0335
Epoch 3/150
138/138 [==============================] - 1s 10ms/step - loss: 0.0138 - accuracy: 0.0599 - val_loss: 0.0135 - val_accuracy: 0.0647
Epoch 4/150
138/138 [==============================] - 1s 8ms/step - loss: 0.0132 - accuracy: 0.0753 - val_loss: 0.0129 - val_accuracy: 0.0818
Epoch 5/150
138/138 [==============================] - 1s 9ms/step - loss: 0.0127 - accuracy: 0.1034 - val_loss: 0.0125 - val_accuracy: 0.1059
Epoch 6/150
138/138 [==============================] - 1s 8ms/step - loss: 0.0125 - accuracy: 0.1189 - val_loss: 0.0124 - val_accuracy: 0.1096
Epoch 7/150
138/138 [==============================] - 1s 8ms/step - loss: 0.0122 - accuracy: 0.1357 - val_loss: 0.0122 - va

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=58 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


SEED:58 Fold:0
Epoch 1/150
138/138 [==============================] - 1s 9ms/step - loss: 0.2358 - accuracy: 0.0260 - val_loss: 0.0171 - val_accuracy: 0.0298
Epoch 2/150
138/138 [==============================] - 1s 7ms/step - loss: 0.0153 - accuracy: 0.0555 - val_loss: 0.0140 - val_accuracy: 0.0834
Epoch 3/150
138/138 [==============================] - 1s 10ms/step - loss: 0.0142 - accuracy: 0.0715 - val_loss: 0.0138 - val_accuracy: 0.0961
Epoch 4/150
138/138 [==============================] - 1s 6ms/step - loss: 0.0137 - accuracy: 0.0960 - val_loss: 0.0131 - val_accuracy: 0.1178
Epoch 5/150
138/138 [==============================] - 1s 7ms/step - loss: 0.0133 - accuracy: 0.1115 - val_loss: 0.0130 - val_accuracy: 0.1314
Epoch 6/150
138/138 [==============================] - 1s 6ms/step - loss: 0.0132 - accuracy: 0.1209 - val_loss: 0.0128 - val_accuracy: 0.1121
Epoch 7/150
138/138 [==============================] - 1s 7ms/step - loss: 0.0128 - accuracy: 0.1344 - val_loss: 0.0127 - val_

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=3255 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


SEED:3255 Fold:0
Epoch 1/150
138/138 [==============================] - 3s 21ms/step - loss: 0.2885 - accuracy: 0.0234 - val_loss: 0.0215 - val_accuracy: 0.0105
Epoch 2/150
138/138 [==============================] - 1s 11ms/step - loss: 0.0167 - accuracy: 0.0413 - val_loss: 0.0145 - val_accuracy: 0.0355
Epoch 3/150
138/138 [==============================] - 1s 8ms/step - loss: 0.0138 - accuracy: 0.0508 - val_loss: 0.0134 - val_accuracy: 0.0551
Epoch 4/150
138/138 [==============================] - 1s 8ms/step - loss: 0.0131 - accuracy: 0.0773 - val_loss: 0.0129 - val_accuracy: 0.1096
Epoch 5/150
138/138 [==============================] - 1s 8ms/step - loss: 0.0127 - accuracy: 0.1033 - val_loss: 0.0125 - val_accuracy: 0.1130
Epoch 6/150
138/138 [==============================] - 1s 8ms/step - loss: 0.0125 - accuracy: 0.1195 - val_loss: 0.0125 - val_accuracy: 0.1424
Epoch 7/150
138/138 [==============================] - 1s 8ms/step - loss: 0.0123 - accuracy: 0.1255 - val_loss: 0.0123 - v

In [37]:
y_pred=final_pred.loc[:,submission.columns[1:]]/15

<h1>TRANSFER LEARNING START</h1>

In [38]:
############################### HPS for transfer learning ##############################################
hps={
    30:[
    {
        "units":[512,1187, 2048, 1971],
        "activations":['elu', 'swish', 'selu','swish'],
        "dropouts":[0.28,0.26,0.47,0.6]
    },
    {
        "units":[1127,532, 1488, 1219],
        "activations":['swish', 'selu','elu','swish'],
        "dropouts":[0.57,0.0,0.16,0.44]
    },
    {
        "units":[1123,640, 512],
        "activations":['elu','swish','elu'],
        "dropouts":[0.14,0.64,0.0]
    },
    {
        "units":[1110,1248,588, 971],
        "activations":['selu','selu','elu','selu'],
        "dropouts":[0.29,0.61,0.64,0.41]
    },
    {
        "units":[1687,1494,1513],
        "activations":['elu','swish','elu'],
        "dropouts":[0.17,0.02,0.66]
    }    
    ],
    58:[
        {
            "units":[1395,1309],
            "activations":['selu','elu'],
            "dropouts":[0.51,0.36]
        },
        {
            "units":[897,1321],
            "activations":['elu', 'swish'],
            "dropouts":[0.59,0.63]
        },
        {
            "units":[1235,841],
            "activations":['selu', 'elu'],
            "dropouts":[0.69,0.05]
        },
        {
            "units":[1628,1272,512],
            "activations":['elu', 'swish', 'elu'],
            "dropouts":[0.63,0.44,0.0]
        },
        {
            "units":[646,1056,1827,774,1463],
            "activations":['selu','elu','swish','selu','selu'],
            "dropouts":[0.51,0.62,0.47,0.35,0.65]
        }
    ],
    3255:[
        {
            "units":[1148,1108,512,512],
            "activations":['selu', 'swish', 'elu', 'elu'],
            "dropouts":[0.28,0.69,0.0,0.0]
        },
        {
            "units":[1060,778,1430],
            "activations":['elu', 'selu', 'elu'],
            "dropouts":[0.68,0.44,0.66]
        },
        {
            "units":[1742,1453,1871,1655],
            "activations":['selu', 'swish', 'swish', 'selu'],
            "dropouts":[0.46,0.57,0.5,0.61]
        },
        {
            "units":[1788,1817,512,512,512,512],
            "activations":[ 'swish', 'elu', 'elu','elu','elu','elu'],
            "dropouts":[0.68,0.54,0.0,0.0,0.0,0.0]
        },
        {
            "units":[907,764],
            "activations":[  'elu', 'elu'],
            "dropouts":[0.47,0.38]
        }
    ]
}

In [39]:
final_pred=submission.iloc[:,1:].copy()
final_pred.loc[:,:]=0

In [40]:
for seed in [30,58,3255]:
    for n,(tr,te) in enumerate(MultilabelStratifiedKFold(n_splits=5, random_state=seed, shuffle=True).split(X_train,y_train)):
        print("SEED:"+str(seed)+" Fold:"+str(n))
        
        units=hps[seed][n]['units']
        activations=hps[seed][n]['activations']
        dropouts=hps[seed][n]['dropouts']
        
        
        model2=create_model(X_train.shape[1], units, activations, dropouts, 1e-3,y_train_non.shape[1])
        
        model2.fit(X_train.values[tr],
         y_train_non.values[tr],
         batch_size=128,
         epochs=150,
         validation_data=(X_train.values[te],y_train_non.values[te]),
         verbose=1,
          callbacks=[
              tf.keras.callbacks.ReduceLROnPlateau(
                  monitor='val_loss', 
                  factor=0.1, 
                  patience=3,
                  epsilon = 1e-4, 
                  mode = 'min',
                  verbose=1
              )
              ,
              
              tf.keras.callbacks.EarlyStopping(
                  monitor='val_loss',
                  min_delta=0,
                  patience=10,
                  mode='auto',
                  verbose=1,
                  baseline=None,
                  restore_best_weights=True
              )
          ]
         )

        model=create_model(X_train.shape[1], units, activations, dropouts, 1e-3, y_train.shape[1])
        for i in range(len(model.layers)-1):
            model.layers[i].set_weights(model2.layers[i].get_weights())

        model.fit(X_train.values[tr],
                 y_train.values[tr],
                 batch_size=128,
                 epochs=150,
                 validation_data=(X_train.values[te],y_train.values[te]),
                 verbose=1,
                  callbacks=[
                      tf.keras.callbacks.ReduceLROnPlateau(
                          monitor='val_loss', 
                          factor=0.1, 
                          patience=3,
                          epsilon = 1e-4, 
                          mode = 'min',
                          verbose=1
                      )
                      ,

                      tf.keras.callbacks.EarlyStopping(
                          monitor='val_loss',
                          min_delta=0,
                          patience=10,
                          mode='auto',
                          verbose=1,
                          baseline=None,
                          restore_best_weights=True
                      )
                  ]
                 )
        model.save(f'tSEED{seed}FOLD{n}.h5')
        model=create_model(X_train.shape[1], units, activations, dropouts, 1e-3, y_train.shape[1])
        model.load_weights(f'tSEED{seed}FOLD{n}.h5')
        
        final_pred+=model.predict(X_test)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=30 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


SEED:30 Fold:0
Epoch 1/150
138/138 [==============================] - 1s 10ms/step - loss: 0.2517 - accuracy: 0.0019 - val_loss: 0.0134 - val_accuracy: 0.0030
Epoch 2/150
138/138 [==============================] - 1s 8ms/step - loss: 0.0110 - accuracy: 0.0033 - val_loss: 0.0091 - val_accuracy: 0.0050
Epoch 3/150
138/138 [==============================] - 1s 10ms/step - loss: 0.0093 - accuracy: 0.0042 - val_loss: 0.0086 - val_accuracy: 0.0062
Epoch 4/150
138/138 [==============================] - 1s 8ms/step - loss: 0.0090 - accuracy: 0.0048 - val_loss: 0.0086 - val_accuracy: 0.0059
Epoch 5/150
138/138 [==============================] - 1s 8ms/step - loss: 0.0089 - accuracy: 0.0054 - val_loss: 0.0085 - val_accuracy: 0.0062
Epoch 6/150
138/138 [==============================] - 1s 8ms/step - loss: 0.0089 - accuracy: 0.0055 - val_loss: 0.0085 - val_accuracy: 0.0071
Epoch 7/150
138/138 [==============================] - 1s 8ms/step - loss: 0.0089 - accuracy: 0.0058 - val_loss: 0.0086 - val

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=58 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


SEED:58 Fold:0
Epoch 1/150
138/138 [==============================] - 1s 7ms/step - loss: 0.2647 - accuracy: 0.0030 - val_loss: 0.0137 - val_accuracy: 0.0018
Epoch 2/150
138/138 [==============================] - 1s 6ms/step - loss: 0.0110 - accuracy: 0.0036 - val_loss: 0.0089 - val_accuracy: 0.0048
Epoch 3/150
138/138 [==============================] - 1s 6ms/step - loss: 0.0091 - accuracy: 0.0048 - val_loss: 0.0084 - val_accuracy: 0.0046
Epoch 4/150
138/138 [==============================] - 1s 6ms/step - loss: 0.0091 - accuracy: 0.0052 - val_loss: 0.0085 - val_accuracy: 0.0048
Epoch 5/150
138/138 [==============================] - 1s 6ms/step - loss: 0.0088 - accuracy: 0.0054 - val_loss: 0.0085 - val_accuracy: 0.0043
Epoch 6/150
138/138 [==============================] - ETA: 0s - loss: 0.0087 - accuracy: 0.0063
Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
138/138 [==============================] - 1s 6ms/step - loss: 0.0087 - accuracy: 0.0063 - v

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=3255 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


SEED:3255 Fold:0
Epoch 1/150
138/138 [==============================] - 1s 9ms/step - loss: 0.2903 - accuracy: 0.0107 - val_loss: 0.0168 - val_accuracy: 0.0018
Epoch 2/150
138/138 [==============================] - 1s 7ms/step - loss: 0.0119 - accuracy: 0.0034 - val_loss: 0.0096 - val_accuracy: 0.0052
Epoch 3/150
138/138 [==============================] - 1s 7ms/step - loss: 0.0089 - accuracy: 0.0038 - val_loss: 0.0085 - val_accuracy: 0.0050
Epoch 4/150
138/138 [==============================] - 1s 8ms/step - loss: 0.0084 - accuracy: 0.0044 - val_loss: 0.0083 - val_accuracy: 0.0046
Epoch 5/150
138/138 [==============================] - 1s 8ms/step - loss: 0.0083 - accuracy: 0.0059 - val_loss: 0.0082 - val_accuracy: 0.0050
Epoch 6/150
138/138 [==============================] - 1s 7ms/step - loss: 0.0082 - accuracy: 0.0058 - val_loss: 0.0082 - val_accuracy: 0.0055
Epoch 7/150
138/138 [==============================] - 1s 7ms/step - loss: 0.0082 - accuracy: 0.0060 - val_loss: 0.0082 - val

In [41]:
y_pred_transfer=final_pred/15
# y_pred=y_pred/2.35

In [42]:
from sklearn.metrics import log_loss
from pytorch_tabnet.metrics import Metric
from sklearn.metrics import roc_auc_score, log_loss

class LogitsLogLoss(Metric):
    """
    LogLoss with sigmoid applied
    """

    def __init__(self):
        self._name = "logits_ll"
        self._maximize = False

    def __call__(self, y_true, y_pred):
        """
        Compute LogLoss of predictions.

        Parameters
        ----------
        y_true: np.ndarray
            Target matrix or vector
        y_score: np.ndarray
            Score matrix or vector

        Returns
        -------
            float
            LogLoss of predictions vs targets.
        """
        logits = 1 / (1 + np.exp(-y_pred))
        aux = (1-y_true)*np.log(1-logits+1e-15) + y_true*np.log(logits+1e-15)
        return np.mean(-aux)

In [43]:
tabnet_params = dict(
    n_d = 32,
    n_a = 32,
    n_steps = 1,
    gamma = 1.3,
    lambda_sparse = 0,
    optimizer_fn = optim.Adam,
    optimizer_params = dict(lr = 2e-2, weight_decay = 1e-5),
    mask_type = "entmax",
    scheduler_params = dict(
        mode = "min", patience = 5, min_lr = 1e-5, factor = 0.9),
    scheduler_fn = ReduceLROnPlateau,
    seed = 456,
    verbose = 10
)

In [44]:
final_pred=submission.iloc[:,1:].copy()
final_pred.loc[:,:]=0

for seed in [30,58,3255]:
    for n,(tr,te) in enumerate(MultilabelStratifiedKFold(n_splits=5, random_state=seed, shuffle=True).split(X_train,y_train)):
        print("SEED:"+str(seed)+" Fold:"+str(n))
        
        model = TabNetRegressor(**tabnet_params)
        
        model.fit(
        X_train = X_train.values[tr],
        y_train = y_train.values[tr],
        eval_set = [(X_train.values[te],y_train.values[te])],
        eval_name = ["val"],
        eval_metric = ["logits_ll"],
        max_epochs = 200,
        patience = 20,
        batch_size = 512, 
        virtual_batch_size = 32,
        num_workers = 1,
        drop_last = False,
        # To use binary cross entropy because this is not a regression problem
        loss_fn = F.binary_cross_entropy_with_logits)
        
 
        model.save_model(f"tabSEED{seed}FOLD{n}")
        
        final_pred+=1 / (1 + np.exp(-model.predict(X_test.values)))

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=30 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


SEED:30 Fold:0
Device used : cuda
epoch 0  | loss: 0.21095 | val_logits_ll: 0.02737 |  0:00:02s
epoch 10 | loss: 0.01829 | val_logits_ll: 0.01803 |  0:00:26s
epoch 20 | loss: 0.01748 | val_logits_ll: 0.01775 |  0:00:49s
epoch 30 | loss: 0.01736 | val_logits_ll: 0.0174  |  0:01:13s
epoch 40 | loss: 0.01712 | val_logits_ll: 0.01748 |  0:01:37s
epoch 50 | loss: 0.0171  | val_logits_ll: 0.01716 |  0:02:00s
epoch 60 | loss: 0.01687 | val_logits_ll: 0.01698 |  0:02:29s
epoch 70 | loss: 0.01668 | val_logits_ll: 0.01689 |  0:02:53s
epoch 80 | loss: 0.01649 | val_logits_ll: 0.01676 |  0:03:16s
epoch 90 | loss: 0.01639 | val_logits_ll: 0.0167  |  0:03:40s
epoch 100| loss: 0.01611 | val_logits_ll: 0.01674 |  0:04:03s
epoch 110| loss: 0.01597 | val_logits_ll: 0.01669 |  0:04:27s

Early stopping occured at epoch 113 with best_epoch = 93 and best_val_logits_ll = 0.01656
Best weights from best epoch are automatically used!
Successfully saved model at tabSEED30FOLD0.zip
SEED:30 Fold:1
Device used : cu

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: overflow encountered in exp


epoch 10 | loss: 0.01817 | val_logits_ll: 0.02084 |  0:00:30s
epoch 20 | loss: 0.01765 | val_logits_ll: 0.01784 |  0:00:54s
epoch 30 | loss: 0.01748 | val_logits_ll: 0.01774 |  0:01:17s
epoch 40 | loss: 0.01717 | val_logits_ll: 0.01722 |  0:01:41s
epoch 50 | loss: 0.01691 | val_logits_ll: 0.0172  |  0:02:05s
epoch 60 | loss: 0.01674 | val_logits_ll: 0.01735 |  0:02:29s
epoch 70 | loss: 0.01669 | val_logits_ll: 0.01709 |  0:02:58s
epoch 80 | loss: 0.0166  | val_logits_ll: 0.01698 |  0:03:21s
epoch 90 | loss: 0.01653 | val_logits_ll: 0.01703 |  0:03:46s
epoch 100| loss: 0.01644 | val_logits_ll: 0.01685 |  0:04:09s
epoch 110| loss: 0.01618 | val_logits_ll: 0.01689 |  0:04:34s
epoch 120| loss: 0.01598 | val_logits_ll: 0.01683 |  0:04:56s
epoch 130| loss: 0.0159  | val_logits_ll: 0.01676 |  0:05:26s
epoch 140| loss: 0.01564 | val_logits_ll: 0.01681 |  0:05:50s
epoch 150| loss: 0.01558 | val_logits_ll: 0.01691 |  0:06:13s
epoch 160| loss: 0.01531 | val_logits_ll: 0.01681 |  0:06:37s

Early s

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=58 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


...

epoch 10 | loss: 0.01828 | val_logits_ll: 0.0182  |  0:00:25s
epoch 20 | loss: 0.01764 | val_logits_ll: 0.01856 |  0:00:49s
epoch 30 | loss: 0.01744 | val_logits_ll: 0.0175  |  0:01:13s
epoch 40 | loss: 0.01709 | val_logits_ll: 0.0173  |  0:01:36s
epoch 50 | loss: 0.0169  | val_logits_ll: 0.01713 |  0:02:00s
epoch 60 | loss: 0.0167  | val_logits_ll: 0.0171  |  0:02:29s
epoch 70 | loss: 0.01647 | val_logits_ll: 0.01704 |  0:02:53s
epoch 80 | loss: 0.01635 | val_logits_ll: 0.01686 |  0:03:17s
epoch 90 | loss: 0.01632 | val_logits_ll: 0.01713 |  0:03:41s
epoch 100| loss: 0.0162  | val_logits_ll: 0.01696 |  0:04:05s
epoch 110| loss: 0.01592 | val_logits_ll: 0.01687 |  0:04:29s
epoch 120| loss: 0.01577 | val_logits_ll: 0.01674 |  0:04:59s
epoch 130| loss: 0.01555 | val_logits_ll: 0.01669 |  0:05:22s
epoch 140| loss: 0.01544 | val_logits_ll: 0.01672 |  0:05:46s
epoch 150| loss: 0.0151  | val_logits_ll: 0.01674 |  0:06:09s
epoch 160| loss: 0.01487 | val_logits_ll: 0.01678 |  0:06:33s

Early s

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=3255 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


SEED:3255 Fold:0
Device used : cuda
epoch 0  | loss: 0.21276 | val_logits_ll: 0.02933 |  0:00:02s


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: overflow encountered in exp


epoch 10 | loss: 0.01837 | val_logits_ll: 0.0184  |  0:00:26s
epoch 20 | loss: 0.01765 | val_logits_ll: 0.01866 |  0:00:50s
epoch 30 | loss: 0.01737 | val_logits_ll: 0.01769 |  0:01:15s
epoch 40 | loss: 0.01734 | val_logits_ll: 0.01744 |  0:01:38s
epoch 50 | loss: 0.01716 | val_logits_ll: 0.01731 |  0:02:07s
epoch 60 | loss: 0.017   | val_logits_ll: 0.01733 |  0:02:32s
epoch 70 | loss: 0.01676 | val_logits_ll: 0.01718 |  0:02:55s
epoch 80 | loss: 0.01669 | val_logits_ll: 0.0171  |  0:03:19s
epoch 90 | loss: 0.0165  | val_logits_ll: 0.01705 |  0:03:43s
epoch 100| loss: 0.0163  | val_logits_ll: 0.01689 |  0:04:07s
epoch 110| loss: 0.01609 | val_logits_ll: 0.01679 |  0:04:37s
epoch 120| loss: 0.01603 | val_logits_ll: 0.01676 |  0:05:00s
epoch 130| loss: 0.01573 | val_logits_ll: 0.01682 |  0:05:24s
epoch 140| loss: 0.0156  | val_logits_ll: 0.01672 |  0:05:47s
epoch 150| loss: 0.01541 | val_logits_ll: 0.01678 |  0:06:12s
epoch 160| loss: 0.01508 | val_logits_ll: 0.01691 |  0:06:36s

Early s

In [45]:
y_pred_tabnet=final_pred/15

In [46]:
y_pred_final=y_pred * 0.25 + y_pred_transfer * 0.25 + y_pred_tabnet * 0.5

In [47]:
submission.head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
1,id_001897cda,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
2,id_002429b5b,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
3,id_00276f245,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
4,id_0027f1083,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5


In [48]:
submission.iloc[:,1:]=y_pred_final

In [49]:
submission.loc[x_test_checkpoint["cp_type"]=='ctl_vehicle','5-alpha_reductase_inhibitor':]=0

In [50]:
submission.head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.000750,0.001091,0.002089,0.019748,0.018584,0.004491,0.002085,0.005966,0.000325,...,0.001072,0.001195,0.004213,0.001195,0.000634,0.000711,0.000899,0.001795,0.005004,0.001838
1,id_001897cda,0.000415,0.000833,0.001801,0.002423,0.001488,0.001938,0.002919,0.012416,0.006070,...,0.000697,0.001371,0.003716,0.000455,0.010757,0.000635,0.020449,0.001069,0.005172,0.003133
2,id_002429b5b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,id_00276f245,0.000664,0.000753,0.002374,0.009352,0.011034,0.003397,0.002577,0.004634,0.000412,...,0.000755,0.002163,0.003602,0.023865,0.008360,0.000626,0.002521,0.002042,0.000779,0.001769
4,id_0027f1083,0.001410,0.001233,0.001858,0.015506,0.019949,0.003358,0.005273,0.002509,0.000525,...,0.000879,0.000934,0.003337,0.001541,0.001119,0.000767,0.001287,0.001722,0.000445,0.001518


In [51]:
submission.sum(1)

0       0.729914
1       0.676417
2       0.000000
3       0.687571
4       0.638297
          ...   
3977    0.740591
3978    0.675896
3979    0.693348
3980    0.711731
3981    0.671265
Length: 3982, dtype: float64

In [52]:
submission.to_csv('submission.csv',index=False)